In [1]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import pandas

from shared_code.utility.spark.set_environ import set_azure_env

set_azure_env()

from shared_code.utility.storage.table import TableAdapter

In [2]:
table_name = "training"

table_adapter: TableAdapter = TableAdapter()

data_path = "/data/parquet/"

parquet_process_data_path = data_path + "processed_data.parquet"

In [3]:
print(f"Reading from parquet {parquet_process_data_path} with Updated Primary Captions")

processed_with_captions = pandas.read_parquet(parquet_process_data_path)

display(processed_with_captions)

Reading from parquet /data/parquet/processed_data.parquet with Updated Primary Captions


,subreddit,file_name,text,thumbnail_path,thumbnail_exists,original_image,original_image_exists,hash,id,original_caption,thumbnail_caption
0,CityPorn,4emw5uldib9a1.jpg,New York in the fog,D:\data\images\CityPorn\thumbnail\4emw5uldib9a...,True,D:\data\images\CityPorn\4emw5uldib9a1.jpg,True,7a8d96e378c15c8ab8440ac311f12c11,1000cej,a city street filled with lots of tall buildings,a city street filled with lots of tall buildings
3,AmIhotAF,4xyb1vgbjb9a1.jpg,Just looking for entertainment,D:\data\images\AmIhotAF\thumbnail\4xyb1vgbjb9a...,True,D:\data\images\AmIhotAF\4xyb1vgbjb9a1.jpg,True,e554c1ed7ffa2740436ac082068b2824,1000glf,a beautiful young woman in a black dress posin...,a beautiful young woman in a black dress posin...
4,greentext,3mewbe0wjb9a1.jpg,Anon wants Elon cut,D:\data\images\greentext\thumbnail\3mewbe0wjb9...,True,D:\data\images\greentext\3mewbe0wjb9a1.jpg,True,1dec3dabb5e46cde01855d06089c287a,1000j1n,a collage of photos showing a man in a suit an...,a collage of photos showing a man in a suit an...
7,sfwpetite,v3ra9g4vrb9a1.jpg,Braids,D:\data\images\sfwpetite\thumbnail\v3ra9g4vrb9...,True,D:\data\images\sfwpetite\v3ra9g4vrb9a1.jpg,True,16ca1536c4f6936d3e994df7cb40ab8e,1001ghp,a beautiful young woman in a bikini posing for...,a beautiful young woman in a bikini posing for...
10,sfwpetite,ldz8tgpsub9a1.jpg,Incredible body,D:\data\images\sfwpetite\thumbnail\ldz8tgpsub9...,True,D:\data\images\sfwpetite\ldz8tgpsub9a1.jpg,True,a0b0ffef01a7abf9de351000026d49f0,1001ssd,a woman in a bikini standing on a beach,a woman in a bikini standing on a beach
...,...,...,...,...,...,...,...,...,...,...,...
8592,SFWRedheads,eu6z40qyya9a1.jpg,Starting the day with mushroom water in my mus...,D:\data\images\SFWRedheads\thumbnail\eu6z40qyy...,True,D:\data\images\SFWRedheads\eu6z40qyya9a1.jpg,True,699ab3d71e8b26f7a72ea73bd2a295a0,zzxyrq,A woman in a pink shirt and black pants is tal...,a woman holding up a bottle of beer
8593,SFWNextDoorGirls,Hzad3q2.jpg,your new years resolution should be sleeping w...,D:\data\images\SFWNextDoorGirls\thumbnail\Hzad...,True,D:\data\images\SFWNextDoorGirls\Hzad3q2.jpg,True,d159090b1c7b485347cec61c70bc9c54,zzy3sq,A girl in a pink dress is talking to another g...,a young woman in a red dress is talking on the...
8594,CityPorn,dwu7i1dni99a1.jpg,"Ronda's bridge is brutal, Spain",D:\data\images\CityPorn\thumbnail\dwu7i1dni99a...,True,D:\data\images\CityPorn\dwu7i1dni99a1.jpg,True,02ae3631262b2de84b0b24c4275deae9,zzy49v,A group of people are standing in front of a l...,a bridge over a body of water with a waterfall
8596,greentext,bgho6WK.jpg,Anon does a little trolling,D:\data\images\greentext\thumbnail\bgho6WK.jpg,True,D:\data\images\greentext\bgho6WK.jpg,True,df666b8b2ad543c77b3fdba89becda1a,zzzeoi,A man wearing a black shirt and black pants is...,a series of photos showing a man in a suit


In [4]:
from typing import List
from dataclasses import dataclass

@dataclass
class DataSources:
	name: str
	data: List[str]

	@staticmethod
	def from_dict(obj: dict) -> 'DataSources':
		_name = obj.get("name")
		_data = [x for x in obj.get("data")]
		return DataSources(_name, _data)

In [5]:
sources = [
	{"name": "CityDiffusion", "data": [
		"CityPorn"
	]},
	{"name": "NatureDiffusion",
	 "data": [
		 "EarthPorn"
	 ]},
	{"name": "MemeDiffusion",
	 "data": [
		 "greentext"
	 ]},
	{"name": "SexyDiffusion",
	 "data": [
		 "HotGirlNextDoor",
		 "sfwpetite",
		 "AmIhotAF",
		 "selfies",
		 "amihot",
		 "SFWNextDoorGirls",
		 "SFWRedheads",
		 "SFWPetite",
		 "Amicute"]
	 }
]

data_sources = [DataSources.from_dict(x) for x in sources]

In [6]:
def process_row_to_line(row):
	_subreddit = row['subreddit']
	for _source in data_sources:
		if _subreddit in source.data:
			return f"<|startoftext|><|model|>{_source.name}<|prompt|>{row['text']}<|text|>{row['original_caption']}<endoftext|>" + "\n"
	return None

In [7]:
lines = []
for record in processed_with_captions.to_dict(orient='records'):
	subreddit = record['subreddit']
	for source in data_sources:
		if subreddit in source.data:
			line = f"<|startoftext|><|model|>{source.name}<|prompt|>{record['text']}<|text|>{record['original_caption']}<|endoftext|>" + "\n"
			lines.append(line)
with open("training.txt", "wb") as f:
	for line in lines:
		f.write(line.encode("utf-8"))

In [8]:
from torch.utils.data import Dataset, random_split
from transformers import GPT2LMHeadModel
from transformers import TrainingArguments, Trainer
import torch
import random

In [9]:
class CustomDataset(Dataset):
	_input_id: str = 'input_ids'
	_attention_mask: str = 'attention_mask'

	def __init__(self, text_list, _tokenizer, _max_length, truncation=False):
		self.input_ids = []
		self.attention_mask = []
		self.labels = []
		for text in text_list:
			encodings_dict = _tokenizer(text, truncation=truncation, max_length=_max_length)
			self.input_ids.append(torch.tensor(encodings_dict[self._input_id]))
			self.attention_mask.append(torch.tensor(encodings_dict[self._attention_mask]))

	def __len__(self):
		return len(self.input_ids)

	def __getitem__(self, index):
		return self.input_ids[index], self.attention_mask[index]

In [10]:
model_type = ""

model_name = f"sd-prompt-bot{model_type}-2"

parent_directory = "/models/"

model_output_dir = f"{parent_directory}/{model_name}"

tokenizer_path = f"{model_output_dir}"

In [11]:
data_lines = []
with open('training.txt', 'r', encoding="UTF-8") as f:
	lines = f.readlines()
	for line in lines:
		foo = line
		print(foo)
		data_lines.append(foo)

random.shuffle(data_lines)

<|startoftext|><|model|>CityDiffusion<|prompt|>New York in the fog<|text|>a city street filled with lots of tall buildings<|endoftext|>

<|startoftext|><|model|>SexyDiffusion<|prompt|>Just looking for entertainment<|text|>a beautiful young woman in a black dress posing for a picture<|endoftext|>

<|startoftext|><|model|>MemeDiffusion<|prompt|>Anon wants Elon cut<|text|>a collage of photos showing a man in a suit and a woman in a suit<|endoftext|>

<|startoftext|><|model|>SexyDiffusion<|prompt|>Braids<|text|>a beautiful young woman in a bikini posing for a picture<|endoftext|>

<|startoftext|><|model|>SexyDiffusion<|prompt|>Incredible body<|text|>a woman in a bikini standing on a beach<|endoftext|>

<|startoftext|><|model|>SexyDiffusion<|prompt|>I heard wearing red on NYE brings good luck... hope it works! 🤞<|text|>a beautiful young woman in a bikini posing for a picture<|endoftext|>

<|startoftext|><|model|>CityDiffusion<|prompt|>Pacific Beach California<|text|>a large body of water wi

In [12]:
from transformers import GPT2Tokenizer

tokenizer = GPT2Tokenizer.from_pretrained(f"gpt2{model_type}")

model = GPT2LMHeadModel.from_pretrained(f"gpt2{model_type}")

special_tokens_dict = {
	"bos_token": "<|startoftext|>",
	"eos_token": "<|endoftext|>",
	"additional_special_tokens": [
		"<|endoftext|>",
		"<|startoftext|>"
	]
}


num_added_toks = tokenizer.add_special_tokens(special_tokens_dict)

print("We have added", num_added_toks, "tokens")

print(f"EOS TOKEN: {tokenizer.eos_token}")
print(f"BOS TOKEN: {tokenizer.bos_token}")
print(f"Additional Special Tokens: {tokenizer.additional_special_tokens}")

print("Resizing Token Embeddings")
model.resize_token_embeddings(len(tokenizer))

print("Saving Model")
model.save_pretrained(model_output_dir)

print("Saving Tokenizer")
tokenizer.save_pretrained(tokenizer_path)

print("Loading Model")
model = GPT2LMHeadModel.from_pretrained(model_output_dir)

print("Loading Tokenizer")
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

We have added 1 tokens
EOS TOKEN: <|endoftext|>
BOS TOKEN: <|startoftext|>
Additional Special Tokens: ['<|endoftext|>', '<|startoftext|>']
Resizing Token Embeddings
Saving Model
Saving Tokenizer
Loading Model
Loading Tokenizer


In [13]:
model.cuda()

generator = torch.Generator()

generator.manual_seed(0)

print(f":: Total Number Of Samples {len(data_lines)}")
max_length = max([len(tokenizer.encode(prompt)) for prompt in data_lines])

print(f":: Max Length Of Sample {max_length}")
dataset = CustomDataset(data_lines, tokenizer, max_length, False)

train_size = int(0.9 * len(dataset))

train_dataset, eval_dataset = random_split(dataset, [train_size, len(dataset) - train_size], generator=generator)

:: Total Number Of Samples 4659
:: Max Length Of Sample 433


In [14]:
training_args = TrainingArguments(output_dir=model_output_dir)
training_args.num_train_epochs = 5
training_args.per_device_train_batch_size = 1
training_args.per_device_eval_batch_size = 1
training_args.logging_steps = 50
training_args.save_steps = 1000
training_args.weight_decay = 0.0
training_args.fp16 = True
training_args.auto_find_batch_size = False
training_args.gradient_accumulation_steps = 1
training_args.learning_rate = 1e-4

In [15]:
trainer: Trainer = Trainer(
	model=model,
	tokenizer=tokenizer,
	args=training_args,
	train_dataset=train_dataset,
	eval_dataset=eval_dataset,
	data_collator=lambda data: {
		'input_ids': torch.stack([x[0] for x in data]),
		'attention_mask': torch.stack([x[1] for x in data]),
		'labels': torch.stack([x[0] for x in data])
	}
)

Using cuda_amp half precision backend


In [16]:
trainer.train()

trainer.save_model(model_output_dir)

D:\workspaces\General\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4193
  Num Epochs = 5
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 1
  Gradient Accumulation steps = 1
  Total optimization steps = 20965
  Number of trainable parameters = 124440576
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as: arzen221. Use `wandb login --relogin` to force relogin


Step,Training Loss



KeyboardInterrupt



In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained(tokenizer_path)

model = GPT2LMHeadModel.from_pretrained(model_output_dir)

In [ ]:

question = "<|startoftext|>"

prompt = f"{question}"

device = torch.device(f"cuda" if torch.cuda.is_available() else "cpu")

generation_prompt = tokenizer(prompt, add_special_tokens=False, return_tensors="pt")

model.to(device)

generation_prompt.to(device)

inputs = generation_prompt.input_ids

attention_mask = generation_prompt['attention_mask']

sample_outputs = model.generate(inputs=inputs,
								attention_mask=attention_mask,
								do_sample=True,
								top_k=0,
								top_p=0.95,
								max_length=1024,
								num_return_sequences=5,
								repetition_penalty=1.1)

result = ""
for i, sample_output in enumerate(sample_outputs):
	result = tokenizer.decode(sample_output, skip_special_tokens=False)
	print(result)